In [ ]:
import pandas as pd
import pymysql
import datetime

### 오키넷 마리아DB 연동 및 데이터 추출

- 전세/매매 데이터에서 우리가 알고자하는 것은 건축물별 금액(월세금액, 보증금액, 거래금액)이다.
- 따라서 건축물의 정보(아파트명, 전용면적, 건축년도)와 거래 금액, 거래 날짜에 관한 변수만 추출하였다.

In [ ]:
# 마리아 DB 연동
conn = pymysql.connect(host = "49.247.147.124", user = "onebiteDb", password = "Onebite!@#456", db = 'onebite' , charset = "utf8")

In [ ]:
# 쿼리문 작성
rent_sql = """
SELECT 건축년도, 년, 월, 일, 법정동, 보증금액, 아파트, 월세금액, 전용면적, 지번, 지역코드
FROM rent
WHERE 지역코드 LIKE '11%';
"""

sales_sql = """
SELECT 건축년도, 년, 월, 일, 법정동, 거래금액, 아파트, 전용면적, 지번, 지역코드, 도로명, 도로명일련번호코드, 도로명건물본번호코드, 도로명건물부번호코드
FROM sales
WHERE 지역코드 LIKE '11%';
"""

In [ ]:
rent_df = pd.read_sql(rent_sql, conn)
sales_df = pd.read_sql(sales_sql, conn)
conn.commit()
conn.close() 

### 전처리

- '보증금액', '월세금액', '거래금액'변수 가격 콤마 제거
- '건축년도'가 없는 데이터 제거 ['건축년도'는 추후에 필터링(혹은 추천 시스템의 파라미터)에 사용되어 개인화 추천에 활용될 변수이므로 결측치있는 row 데이터를 제거하였다.]
- DB의 모든 변수가 문자열의 형태를 가지고 있었다. 따라서 숫자 형태로 구성되야하는 변수들에 대해 형변환을 진행하였다. 

In [ ]:
# 가격 콤마 제거
rent_df['보증금액'] = rent_df['보증금액'].apply(lambda x: int(''.join(x.split(','))))
rent_df['월세금액'] = rent_df['월세금액'].apply(lambda x: int(''.join(x.split(','))))

sales_df['거래금액'] = sales_df['거래금액'].apply(lambda x: int(''.join(x.split(','))))

# 건축년도 노이즈 제거
rent_df = rent_df.dropna(subset = ['건축년도'], axis = 0)

sales_df = sales_df.dropna(subset = ['건축년도'], axis = 0)

# 문자열 -> 숫자형 데이터 형변환
int_type_list = ['건축년도', '년', '월', '일', '보증금액', '월세금액']
for col in int_type_list :
    rent_df[col] = rent_df[col].astype(int)
rent_df['전용면적'] = rent_df['전용면적'].astype(float)

int_type_list = ['건축년도', '년', '월', '일', '거래금액']
for col in int_type_list :
    sales_df[col] = sales_df[col].astype(int)
sales_df['전용면적'] = sales_df['전용면적'].astype(float)


####refoactor

int_type_list = ['건축년도', '년', '월', '일', '보증금액', '월세금액']
rent_df[int_type_list] = rent_df[int_type_list].astype(int)
rent_df['전용면적'] = rent_df['전용면적'].astype(float)


int_type_list = ['건축년도', '년', '월', '일', '거래금액']
sales_df[int_type_list] = sales_df[int_type_list].astype(int)
sales_df['전용면적'] = sales_df['전용면적'].astype(float)


### Feature Engineering

- 데이터 통합 : 전세/매매 데이터를 '아파트', '전용면적'으로 묶어서 row 데이터를 거래별이 아닌 건축물별로 변환
- 파생변수 추가_1 : 월세/전세/매매에 따른 '평균금액', '최근 3년간 평균금액' 변수 추가

- 파생변수 추가_2 : '연식', '전세가율', '환산보증금' 변수 추가

In [ ]:
# 건축물을 기준으로 데이터 통합
rent_apt = rent_df.groupby(['아파트', '전용면적']).nth(0).reset_index()
sales_apt = sales_df.groupby(['아파트','전용면적']).nth(0).reset_index()

# 작업에 사용할 df
temp_rent_m_df = rent_df[rent_df['월세금액'] != 0].copy() # 월세
temp_rent_c_df = rent_df[rent_df['월세금액'] == 0].copy() # 전세
temp_sales_df = sales_df.copy()


####refactor

rent_apt = rent_df.groupby(['아파트', '전용면적']).first().reset_index()
sales_apt = sales_df.groupby(['아파트', '전용면적']).first().reset_index()

temp_rent_m_df = rent_df[rent_df['월세금액'] != 0].copy() # 월세
temp_rent_c_df = rent_df[rent_df['월세금액'] == 0].copy() # 전세
temp_sales_df = sales_df.copy()


#### 파생변수 추가

In [ ]:
def get_3yr_price(df, today) :
    within_3yr = []

    for yy, mm, dd in zip(df['년'], df['월'], df['일']) :
        if (today - datetime.date(yy, mm, dd)).days < 365*3 :
            within_3yr.append((today - datetime.date(yy, mm, dd)).days)
        else :
            within_3yr.append(False)
    
    df['within_3yr'] = within_3yr
    return df

### refactor

def get_3yr_price(df, today):
    within_3yr = ((today - pd.to_datetime(df[['년', '월', '일']])).dt.days < 365*3)
    df['within_3yr'] = within_3yr
    return df


In [ ]:
# 평균금액
rent_c_guarantee_price = temp_rent_c_df.groupby(['아파트', '전용면적'])['보증금액'].mean().round(3).reset_index(name='평균전세보증금액')
rent_m_guarantee_price = temp_rent_m_df.groupby(['아파트', '전용면적'])['보증금액'].mean().round(3).reset_index(name='평균월세보증금액')
rent_monthly_price = temp_rent_m_df.groupby(['아파트', '전용면적'])['월세금액'].mean().round(3).reset_index(name='평균월세금액')

sales_trans_price = temp_sales_df.groupby(['아파트', '전용면적'])['거래금액'].mean().round(3).reset_index(name='평균거래금액')

# 최근 3년 평균금액
today = datetime.date.today()

temp_rent_c_df = get_3yr_price(temp_rent_c_df, today)
temp_rent_m_df = get_3yr_price(temp_rent_m_df, today)
temp_sales_df = get_3yr_price(temp_sales_df, today)

temp_rent_c_df = temp_rent_c_df[temp_rent_c_df['within_3yr'] != False].reset_index()
temp_rent_m_df = temp_rent_m_df[temp_rent_m_df['within_3yr'] != False].reset_index()
temp_sales_df = temp_sales_df[temp_sales_df['within_3yr'] != False].reset_index()

rent_c_gp_3yr = temp_rent_c_df.groupby(['아파트', '전용면적'])['보증금액'].mean().round(3).reset_index(name='최근3년_전세보증금액')
rent_m_gp_3yr = temp_rent_m_df.groupby(['아파트', '전용면적'])['보증금액'].mean().round(3).reset_index(name='최근3년_월세보증금액')
rent_m_3yr = temp_rent_m_df.groupby(['아파트', '전용면적'])['월세금액'].mean().round(3).reset_index(name='최근3년_월세금액')

sales_trans_3yr = temp_sales_df.groupby(['아파트', '전용면적'])['거래금액'].mean().round(3).reset_index(name='최근3년_거래금액')

In [ ]:
# '평균금액', '최근 3년 평균금액' 파생변수 추가
rent_apt = rent_apt.merge(rent_c_guarantee_price, how='left')
rent_apt = rent_apt.merge(rent_m_guarantee_price, how='left')
rent_apt = rent_apt.merge(rent_monthly_price, how='left')

rent_apt = rent_apt.merge(rent_c_gp_3yr, how='left')
rent_apt = rent_apt.merge(rent_m_gp_3yr, how='left')
rent_apt = rent_apt.merge(rent_m_3yr, how='left')

sales_apt = sales_apt.merge(sales_trans_price, how='left')
sales_apt = sales_apt.merge(sales_trans_3yr, how='left')


###refactor

rent_apt = (
    rent_apt.merge(rent_c_guarantee_price, how='left')
             .merge(rent_m_guarantee_price, how='left')
             .merge(rent_monthly_price, how='left')
             .merge(rent_c_gp_3yr, how='left')
             .merge(rent_m_gp_3yr, how='left')
             .merge(rent_m_3yr, how='left')
)

sales_apt = (
    sales_apt.merge(sales_trans_price, how='left')
             .merge(sales_trans_3yr, how='left')
)



#### 불필요한 변수 제거

In [ ]:
sales_apt = sales_apt.drop(['거래금액'], axis = 1)

rent_apt = rent_apt.drop(['보증금액', '월세금액'], axis = 1)

### 전세/매매 df 통합

In [ ]:
fill_col_list = ['건축년도', '년', '월', '일', '법정동', '지번', '지역코드', '연식']

def fill_nan (df) :
    for col in fill_col_list :
        df[col + '_y'] = df[col + '_y'].fillna(df[col + '_x'])
        df[col + '_x'] = df[col + '_x'].fillna(df[col + '_y'])

        df = df.drop([col + '_y'], axis = 1)
        df = df.rename(columns = {col + '_x' : col})
    return df

In [ ]:
apt = sales_apt.merge(rent_apt, on=['아파트','전용면적'],how='outer')
apt = fill_nan(apt)

#### 파생변수 추가_2

In [ ]:
# 연식
apt['연식'] = sales_apt.apply(lambda row: today.year - row['건축년도'], axis=1)

# 전세가율
apt['전세가율'] = (apt['평균전세보증금액'] / apt['평균거래금액']) * 100

# 환산보증금
apt['환산보증금'] = apt['평균월세보증금액'] + apt['평균월세금액']

### 저장

In [ ]:
output_path = 'C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite/00_rent_sales/output_data/'

apt.to_csv(output_path + 'transaction_db.csv')